<a href="https://colab.research.google.com/github/rahiakela/small-language-models-fine-tuning/blob/main/domain-specific-small-language-models/07-advanced-quantization-techniques/02_model_quantization_with_smoothquant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using SmoothQuant on OPT large models


The code in this notebook is to show evidence that for LLMs having more 6 or more billion parameters, systematic outliers in a model's activations lead to a degradation in accuracy after quantization, and that the application of the [SmoothQuant](https://github.com/mit-han-lab/smoothquant) technique mitigates that risk. While the code refers to the Meta AI's [OPT 6.7 B](https://huggingface.co/facebook/opt-6.7b) model, the same applies to other models too. It requires hardware acceleration to be executed.  

## Setup

We need to force the upgrade of the HF's Datasets library to the latest version. Restart the runtime at the end of this upgrade and before moving on with other cells code execution.

In [ ]:
!pip install --force-reinstall datasets

Install SmoothQuant from source.

In [ ]:
!pip install git+https://github.com/mit-han-lab/smoothquant.git

In [ ]:
!pip install transformers -U

Import the required dependencies.

In [3]:
import torch
from transformers.models.opt.modeling_opt import OPTAttention, OPTDecoderLayer, OPTForCausalLM
from transformers import GPT2Tokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import W8A8Linear

Define a custom finction to quantize a model (weights and activations) in INT8 precision.

In [4]:
def quantize_model(model, weight_quant='per_tensor', act_quant='per_tensor', quantize_bmm_input=True):
    for name, m in model.model.named_modules():
        if isinstance(m, OPTDecoderLayer):
            m.fc1 = W8A8Linear.from_float(m.fc1, weight_quant=weight_quant,
                                          act_quant=act_quant)
            m.fc2 = W8A8Linear.from_float(m.fc2, weight_quant=weight_quant,
                                          act_quant=act_quant)
        elif isinstance(m, OPTAttention):
            m.q_proj = W8A8Linear.from_float(
                m.q_proj, weight_quant=weight_quant, act_quant=act_quant,
                quantize_output=quantize_bmm_input)
            m.k_proj = W8A8Linear.from_float(
                m.k_proj, weight_quant=weight_quant, act_quant=act_quant,
                quantize_output=quantize_bmm_input)
            m.v_proj = W8A8Linear.from_float(
                m.v_proj, weight_quant=weight_quant, act_quant=act_quant,
                quantize_output=quantize_bmm_input)
            m.out_proj = W8A8Linear.from_float(m.out_proj,
                                               weight_quant=weight_quant, act_quant=act_quant)
    return model

Implementa a class to evaluate an LLM given a test dataset.

In [5]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        def tokenize_function(examples):
            example = self.tokenizer(examples['text'])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type='torch', columns=['input_ids'])

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        total, hit = 0, 0
        for batch in self.dataset:
            input_ids = batch['input_ids'].to(self.device).unsqueeze(0)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            last_token_logits = outputs.logits[:, -2, :]
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
        acc = hit / total
        return acc

## Load model

Download a subset (1000 samples in this case) of the LAMBADA dataset and the Meta AI OPT 6.7B model's tokenizer from the Hugging Face's Hub and then create an instance of the Evaluator class using them. Everything goes to GPU.

In [6]:
from datasets import load_dataset
from transformers import GPT2Tokenizer
import torch

model_id = 'facebook/opt-6.7b'
tokenizer = GPT2Tokenizer.from_pretrained(model_id)

# Load a dataset for the evaluation of the different versions of the target model
dataset = load_dataset('cimec/lambada', split='validation[:1000]')

In [7]:
evaluator = Evaluator(dataset, tokenizer, 'cuda')
print("Dataset loaded and Evaluator initialized successfully.")

Dataset loaded and Evaluator initialized successfully.


## FP16 Model Accuracy

Download the Meta AI OPT 6.7B model in FP16 from the HF's Hub.

In [6]:
model_fp16 = OPTForCausalLM.from_pretrained(model_id,
                                            torch_dtype=torch.float16,
                                            device_map='auto',
                                            offload_folder='.')
model_fp16.eval()

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear(in_features=16384, out_features=4096, bias=True)
          (final_layer_norm): Laye

Evaluate the model on the 1000 samples from the LAMBADA dataset.

In [7]:
acc_fp16 = evaluator.evaluate(model_fp16)
print(f'Original model (fp16) accuracy: {acc_fp16}')

Original model (fp16) accuracy: 0.798


## Naive W8A8 Quantized Model Accuracy

Quantize weights and activation of the vanilla model (no SmoothQuant).

In [8]:
# Let’s quantize this model now
model_w8a8 = quantize_model(model_fp16)

In [9]:
print(model_w8a8)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, element

Evaluate the quantized model on the 1000 samples from the LAMBADA dataset.

In [10]:
acc_w8a8 = evaluator.evaluate(model_w8a8)
print(f'Naive W8A8 quantized model accuracy: {acc_w8a8}')

Naive W8A8 quantized model accuracy: 0.423


## SmoothQuant W8A8 Quantized Model Accuracy

**To save time and free GPU memory to evaluate the model after applying SmoothQuant, a runtime restart is recommended at this time, before proceeding further.**

Download the specific model's scales from the HF's Hub (mandatory to apply SmoothQuant).

In [ ]:
!mkdir ./act_scales
%cd act_scales
!wget https://huggingface.co/mit-han-lab/smoothquant-scales/resolve/main/opt-6.7b.pt
%cd ..

Apply SmoothQuant and after quantize the vanilla model's weights and activations in INT8 format.

In [ ]:
model_fp16 = OPTForCausalLM.from_pretrained(model_id,
                                            torch_dtype=torch.float16,
                                            device_map='auto',
                                            offload_folder='.')

In [9]:
act_scales = torch.load('./act_scales/opt-6.7b.pt')
smooth_lm(model_fp16, act_scales, 0.5)
model_smoothquant_w8a8 = quantize_model(model_fp16)
print(model_smoothquant_w8a8)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, element

Evaluate the smooth quantized model on the 1000 samples from the LAMBADA dataset.

In [10]:
model_smoothquant_w8a8.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(4096, 4096, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, element

In [11]:
acc_smoothquant_w8a8 = evaluator.evaluate(model_smoothquant_w8a8)
print(f'SmoothQuant W8A8 quantized model accuracy: {acc_smoothquant_w8a8}')

SmoothQuant W8A8 quantized model accuracy: 0.799


The accuracy of the vanilla model and its smooth quantized version should be comparable, while there should be a significant drop (up to 40%) for the naive quantized model.